In [22]:
ls /home/gsm/Documents/extras/kaggle/petfinder/

breed_labels.csv  color_labels.csv  state_labels.csv  test/  train/


In [179]:
#cat /home/gsm/Documents/extras/kaggle/petfinder/state_labels.csv

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [190]:
df = pd.read_csv('/home/gsm/Documents/extras/kaggle/petfinder/train/train.csv')
drop_names = ['RescuerID','FurLength','Name','Color3','PetID']
ddf = df.drop(columns=drop_names)

In [191]:
ddf.head(3)

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,MaturitySize,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,Description,PhotoAmt,AdoptionSpeed
0,2,3,299,0,1,1,7,1,2,2,2,1,1,100,41326,0,Nibble is a 3+ month old ball of cuteness. He ...,1.0,2
1,2,1,265,0,1,1,2,2,3,3,3,1,1,0,41401,0,I just found it alone yesterday near my apartm...,2.0,0
2,1,1,307,0,1,2,7,2,1,1,2,1,1,0,41326,0,Their pregnant mother was dumped by her irresp...,7.0,3


In [373]:
#TODO clean up Descriptions nan's with  some imputer in text pipeline
ddf = ddf.dropna()

In [374]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import Binarizer,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder,QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score



In [380]:
numeric_features = ['Age']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])

categorical_features = ['Type']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

text_features = 'Description'
text_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('bow', CountVectorizer())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('txt', text_transformer,text_features)],
    remainder='drop')

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RFC())])

X = ddf.drop('AdoptionSpeed', axis=1)
y = ddf['AdoptionSpeed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True)

clf.fit(X_train, y_train)

/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbo...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [381]:
yh = clf.predict(X_test)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.364


In [382]:
cohen_kappa_score(y_test,yh)

0.15426895837623

In [383]:
print(classification_report(yh,y_test))

              precision    recall  f1-score   support

           0       0.07      0.29      0.11        21
           1       0.32      0.30      0.31       650
           2       0.39      0.33      0.36       962
           3       0.28      0.37      0.32       491
           4       0.46      0.45      0.45       873

   micro avg       0.36      0.36      0.36      2997
   macro avg       0.30      0.35      0.31      2997
weighted avg       0.38      0.36      0.37      2997

